In [ ]:
import numpy as np
from qiskit.circuit.library import TwoLocal
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2, Session
from scipy.optimize import minimize
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_algorithms import NumPyMinimumEigensolver
import matplotlib.pyplot as plt

In [ ]:
print("=" * 60)
print("VQE for N2 Molecule on IBM Quantum Computer")
print("=" * 60)

# ---- Step 1: Generate N2 Hamiltonian ----
print("\n[1/5] Generating N2 Hamiltonian...")
driver = PySCFDriver(atom='N 0 0 0; N 0 0 1.098', basis='sto3g')
problem = driver.run()
mapper = JordanWignerMapper()
hamiltonian = mapper.map(problem.hamiltonian.second_q_op())

print(f"  ✓ Number of qubits: {hamiltonian.num_qubits}")
print(f"  ✓ Number of Pauli terms: {len(hamiltonian)}")

In [ ]:
# ---- Step 3: Create Ansatz Circuit ----
print("\n[3/5] Creating ansatz circuit...")
# Use fewer reps for real hardware to reduce circuit depth
ansatz = TwoLocal(
    num_qubits=hamiltonian.num_qubits,
    rotation_blocks=["ry", "rz"],
    entanglement_blocks="cz",
    entanglement="linear",
    reps=1,  # Start with 1 rep, increase if needed
    insert_barriers=False  # Remove barriers for shorter circuits
)

print(f"  ✓ Number of parameters: {ansatz.num_parameters}")
print(f"  ✓ Circuit depth (before transpilation): {ansatz.decompose().depth()}")

# Initialize parameters (can use zeros or small random values)
#initial_params = np.random.uniform(-0.1, 0.1, ansatz.num_parameters)
initial_params = [0 for _ in range(ansatz.num_parameters)]

In [ ]:
# ---- Step 4: Setup IBM Quantum Backend ----
print("\n[4/5] Connecting to IBM Quantum...")
service = QiskitRuntimeService()

# Option 1: Use least busy backend
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=hamiltonian.num_qubits)

# Option 2: Specify a particular backend (uncomment to use)
# backend = service.backend('ibm_brisbane')  # or 'ibm_kyoto', 'ibm_osaka', etc.

print(f"  ✓ Selected backend: {backend.name}")
print(f"  ✓ Number of qubits: {backend.num_qubits}")
print(f"  ✓ Pending jobs: {backend.status().pending_jobs}")


In [ ]:
# ---- Step 5: Run VQE on QPU ----
print("\n[5/5] Running VQE on IBM Quantum hardware...")
print("  (This may take 15-30 minutes depending on queue)")
print("-" * 60)

cost_history = []
iteration_count = [0]

# Initialize estimator with backend
estimator = EstimatorV2(mode=backend)

# Configure estimator options (set once, outside the cost function)
#estimator.options.resilience_level = 1  # Error mitigation
estimator.options.default_shots = 4096  # Number of shots per circuit

def cost_function_qpu(params):
    """Cost function for VQE optimization on QPU"""
    iteration_count[0] += 1
    
    # Bind parameters to ansatz
    bound_circuit = ansatz.assign_parameters(params)
    
    # Transpile the bound circuit for the backend
    from qiskit import transpile
    isa_circuit = transpile(
        bound_circuit,
        backend=backend,
        optimization_level=3,
       
       # layout_method='dense',  # Use dense layout to keep qubits together
       # routing_method='sabre'
    )
    
    # Print transpiled circuit info on first iteration
    if iteration_count[0] == 1:
        print(f"  ✓ Transpiled circuit depth: {isa_circuit.depth()}")
        print(f"  ✓ Transpiled circuit gates: {isa_circuit.count_ops()}")
        print(f"  ✓ Circuit qubits: {isa_circuit.num_qubits}")
        print(f"  ✓ Physical qubits used: {isa_circuit.layout.final_index_layout()[:hamiltonian.num_qubits]}")
    
    # Get the qubit mapping from the transpiled circuit
    qubit_mapping = isa_circuit.layout.final_index_layout()
    
    # Apply the Hamiltonian to the same physical qubits as the circuit
    # Expand Hamiltonian to match the circuit's qubit count
    from qiskit.quantum_info import SparsePauliOp
    
    # Create identity for unused qubits
    identity_qubits = isa_circuit.num_qubits - hamiltonian.num_qubits
    
    if identity_qubits > 0:
        # Extend each Pauli string with identities
        extended_paulis = []
       # extended_coeffs = []
        extended_coeffs = hamiltonian.coeffs
        for pauli_str, coeff in zip(hamiltonian.paulis, hamiltonian.coeffs):
            # Add identities to match circuit size
            extended_pauli = str(pauli_str) + 'I' * identity_qubits
            extended_paulis.append(extended_pauli)
            #extended_coeffs.append(coeff)
        
        extended_hamiltonian = SparsePauliOp(extended_paulis, extended_coeffs)
    else:
        extended_hamiltonian = hamiltonian
    
    # Create PUB with extended Hamiltonian
    pub = (isa_circuit, extended_hamiltonian)
    
    # Submit job to quantum computer
    job = estimator.run([pub])
    result = job.result()
    energy = result[0].data.evs
    
    cost_history.append(energy)
    error_str = ""
    if exact_energy is not None:
        error = abs(energy - exact_energy)
        error_str = f" | Error: {error:.6f} Ha"
    
    print(f"  Iter {iteration_count[0]:2d}: Energy = {energy:.8f} Ha{error_str}")
    
    return energy


In [ ]:
print(f"\nStarting optimization")

result = minimize(
    cost_function_qpu,
    initial_params,
    method="COBYLA",  # Derivative-free optimizer
    # options={
    #     "tol": 1e-4         # Convergence tolerance
    # }
)

In [ ]:
exact_energy = -109.282  # Hartree-Fock energy for N2 in STO-3G basis
# ---- Results Summary ----
print("\n" + "=" * 60)
print("VQE RESULTS")
print("=" * 60)
print(f"Final VQE energy:    {result.fun:.8f} Ha")
if exact_energy is not None:
    print(f"Exact energy:        {exact_energy:.8f} Ha")
    error = abs(result.fun - exact_energy)
    print(f"Absolute error:      {error:.8f} Ha ({error*627.5:.4f} kcal/mol)")
    print(f"Relative error:      {error/abs(exact_energy)*100:.4f}%")
print(f"Optimization status: {result.message}")
print(f"Total iterations:    {len(cost_history)}")
print(f"Function evals:      {result.nfev}")

# ---- Plot Convergence ----
print("\nGenerating convergence plot...")
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cost_history) + 1), cost_history, 'bo-', linewidth=2, markersize=8)
if exact_energy is not None:
    plt.axhline(y=exact_energy, color='r', linestyle='--', linewidth=2, label='Exact Energy')
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Energy (Hartree)', fontsize=12)
plt.title('VQE Convergence on IBM Quantum Hardware (N₂ Molecule)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.savefig('n2_vqe_convergence.png', dpi=300)
print("  ✓ Plot saved as 'n2_vqe_convergence.png'")
plt.show()

print("\n" + "=" * 60)
print("VQE COMPLETE!")
print("=" * 60)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    instance='crn:v1:bluemix:public:quantum-computing:us-east:a/2c7afd2718fe41cf912e8ad6e77f57c7:93a5135f-4cde-49be-95a4-c414fe7c8cde::'
)
job = service.job('d4mca7iv0j9c73e65u60')
job_result = job.result()

for idx, pub_result in enumerate(job_result):
    print(f"Expectation values for pub {idx}: {pub_result.data.evs}")